# Exercise: computing the distance at which a projectile will hit the ground

**Author: Giovanni Pizzi, EPFL**

In this exercise, you are given three parameters, defining the initial conditions at which a projectile is launched. 
In particular, you are given in input:

- the height $h$ above the ground from which the projectile is launched
- the two components (horizontal $v_x$ and vertical $v_y$) of the velocity, $\vec v = (v_x, v_y)$ at which the projectile is launched

![Simple schematic](./explanation.png)

## Task
**Your task is to write a python function that, given these three parameters, computes the horizontal position $D$ at which the projectile will hit the ground.**

## How to test the results
To test your function, you can move the sliders below that determine the initial conditions of the projectile.

A real-time visualization will show the correct solution for the problem (solid curve), where the launch point is marked by a black dot and the correct hitting point by a black cross.

You will also see the result of your proposed solution as a large red circle. Finally, You can inspect possible errors of your function by opening the tab "Results of the validation of your function".

In [ ]:
%matplotlib notebook
import numpy as np
import pylab as pl

import tabulate
from ipywidgets import Label, Button, Output, FloatSlider, HBox, VBox, Layout, HTML, Accordion
from widget_code_input import WidgetCodeInput
from IPython.display import display

In [ ]:
# Value of the vertical (downwards) acceleration
g = 9.81 # m/s^2

In [ ]:
code_widget = WidgetCodeInput(
    function_name="get_hit_coordinate", 
    function_parameters="vertical_position, horizontal_v, vertical_v, g={}".format(g),
    docstring="""
A function to compute the hit coordinate of a projectile 
on the ground, knowing the initial launch parameters.

:param vertical_position: launch vertical position [m]
:param horizontal_v: launch horizontal position [m/s]
:param vertical_v: launch vertical position [m/s] 
    (positive values means upward velocity)
:param g: the vertical (downwards) acceleration (default: Earth's gravity)
    
:return: the position at which the projectile will hit the ground [m]
""",
    function_body="# Input here your solution\n# After changing the function, move one of the sliders to validate your function")
display(code_widget)

## The solution:
# import math
# return horizontal_v * (vertical_v + math.sqrt(vertical_v**2 + 2. * g * vertical_position)) / g

In [ ]:
vertical_position_widget = FloatSlider(
    value=6, min=0, max=10, 
    description="Vertical position [m]",
    continuous_update=False, 
    style={'description_width': 'initial'}, layout=Layout(width='50%', min_width='350px'))
horizontal_v_widget = FloatSlider(
    value=5, min=-10, max=10, 
    description="Horizontal velocity [m/s]",
    continuous_update=False, 
    style={'description_width': 'initial'}, layout=Layout(width='50%', min_width='350px'))
vertical_v_widget = FloatSlider(
    value=3, min=-10, max=10, 
    description="Vertical velocity [m/s]",
    continuous_update=False, 
    style={'description_width': 'initial'}, layout=Layout(width='50%', min_width='350px'))

plot_box = Output()

input_box = VBox([vertical_position_widget, horizontal_v_widget, vertical_v_widget])
display(HBox([input_box, plot_box]))

In [ ]:
check_function_output = Output()
check_accordion = Accordion(children=[check_function_output], selected_index=None)
check_accordion.set_title(0, 'Results of the validation of your function (click here to see them)')

display(check_accordion)

In [ ]:
def trajectory(t, vertical_position, horizontal_v, vertical_v, g):
    """
    Return the coordinates (x, y) at time t
    """
    # We define the initial x coordinate to be zero
    x0 = 0
        
    x = x0 + horizontal_v * t
    y = -0.5 * g* t**2 + vertical_v * t + vertical_position
    
    return x, y    

def hit_conditions(vertical_position, horizontal_v, vertical_v, g):
    """
    Return (t, D), where t is the time at which the ground is hit, and D 
    is the distance at which the projectile hits the ground
    """
    
    # We define the initial x coordinate to be zero
    x0 = 0
    
    # x = x0 + horizontal_v * t => t = (x-x0) / horizontal_v
    # y = -0.5 * g* t**2 + vertical_v * t + vertical_position => 
    #
    # y == 0 => 
    a = -0.5 * g
    b = vertical_v
    c = vertical_position
    
    # the two solutions; I want the solution with positive t, 
    # that will in any case be t1, because
    # t1 > t2 for any value of a, b, c (since a < 0)
    t1 = (-b - np.sqrt(b**2 - 4 * a * c)) / (2. * a)
    #t2 = (-b + np.sqrt(b**2 - 4 * a * c)) / (2. * a)
    
    t = t1
    
    D = x0 + horizontal_v * t
    
    return t, D


In [ ]:
def check_user_value():
    # I don't catch exceptions so that the users can see the traceback
    error_string = "YOUR FUNCTION DOES NOT SEEM RIGHT, PLEASE TRY TO FIX IT"
    ok_string = "YOUR FUNCTION SEEMS TO BE CORRECT!! CONGRATULATIONS!"
    
    test_table = []
    last_exception = None
    type_warning = False
    
    check_function_output.clear_output(wait=True)
    with check_function_output:
        user_function = code_widget.get_function_object() 

        test_values_vpos = range(1,7)
        test_values_vx = range(-2,3)
        test_values_vy = range(-2,3)            
        for test_vpos in test_values_vpos:
            for test_vx in test_values_vx:
                for test_vy in test_values_vy:
                    correct_value = hit_conditions(vertical_position=test_vpos, 
                        horizontal_v=test_vx,
                        vertical_v=test_vy,
                        g=g
                    )[1] # [1] because this gives D ([0] is instead t_hit)
                    try:
                        user_hit_position = user_function(
                            vertical_position=test_vpos, 
                            horizontal_v=test_vx,
                            vertical_v=test_vy,
                        )
                        try:
                            error = abs(user_hit_position - correct_value)
                        except Exception:
                            type_warning = True
                            error = 1. # Large value so it triggers a failed test
                    except Exception as exc:
                        last_exception = exc
                        test_table.append([test_vpos, test_vx, test_vy, correct_value, "ERROR", False])
                    else:
                        if error > 1.e-8:
                            test_table.append([test_vpos, test_vx, test_vy, str(correct_value), str(user_hit_position), False])
                        else:
                            test_table.append([test_vpos, test_vx, test_vy, str(correct_value), str(user_hit_position), True])

        num_tests = len(test_table)
        num_passed_tests = len([test for test in test_table if test[5]])
        failed_tests = [test[:-1] for test in test_table if not test[5]] # Keep only failed tests, and remove last column
        MAX_FAILED_TESTS = 5
        if num_passed_tests < num_tests:
            html_table = HTML("<style>tbody tr:nth-child(odd) { background-color: #e2f7ff; } th { background-color: #94cae0; min-width: 100px; } td { font-family: monospace; } td, th { padding-right: 3px; padding-left: 3px; } </style>" + 
                             tabulate.tabulate(
                                 failed_tests[:MAX_FAILED_TESTS], 
                                 tablefmt='html',
                                 headers=["vertical_position", "horizontal_v", "vertical_v", "Expected value", "Your value"]
                             ))
                
        if num_passed_tests < num_tests:
            print("Your function does not seem correct; only {}/{} tests passed".format(num_passed_tests, num_tests))
            print("Printing up to {} failed tests:".format(MAX_FAILED_TESTS))
            display(html_table)
        else:
            print("Your function is correct! Very good! All {} tests passed".format(num_tests))
        
        if type_warning:
            print("WARNING! in at least one case, your function did not return a valid float number, please double check!".format(num_tests))
    
        # Raise the last exception obtained
        if last_exception is not None:
            print("I obtained at least one exception")
            raise last_exception from None
                        
def get_user_value():
    """
    This function returns the value computed by the user's
    function for the current sliders' value, or None if there is an exception
    """
    with check_function_output:
        user_function = code_widget.get_function_object() 
        try:
            user_hit_position = user_function(
                vertical_position=vertical_position_widget.value, 
                horizontal_v=horizontal_v_widget.value,
                vertical_v=vertical_v_widget.value,
            )
        except Exception as exc:
            return None
    return user_hit_position        

In [ ]:
with plot_box:
    the_figure = pl.figure(figsize=(4,3))
    the_plot = the_figure.add_subplot(1,1,1)
    the_plot.set_xlabel("x [m]")
    the_plot.set_xlabel("y [m]")

def replot(vertical_position, horizontal_v, vertical_v):
    global the_plot, g
    
    # Compute correct values
    t_hit, D = hit_conditions(vertical_position, horizontal_v, vertical_v, g)
    t_array = np.linspace(0,t_hit, 100)
    x_array, y_array = trajectory(t_array, vertical_position, horizontal_v, vertical_v, g)

    # Clean up the graph
    the_plot.axes.clear()
    # Plot orrect curves and points
    the_plot.plot([0], [vertical_position], 'ok')
    the_plot.plot([D], [0], 'xk')    
    the_plot.plot(x_array, y_array, '-b')

    
    ## (Try to) plot user value
    user_value = None
    try:
        user_value = get_user_value()
    except Exception:
        # Just a guard not to break the visualization, we should not end up here
        pass 
    try:
        if user_value is not None:
            the_plot.plot([user_value], [0], 'or')    
    except Exception:
        # We might end up here if the function does not return a float value
        pass 

    the_plot.axhline(0, color='gray')
    # Set zoom to fixed value
    the_plot.set_xlim([-30, 30])
    the_plot.set_ylim([-1, 16])
    
    # Redraw
    the_figure.canvas.draw()
    the_figure.canvas.flush_events()

In [ ]:
def recompute(e):
    global the_plot, g
    
    if e is not None:
        if e['type'] != 'change' or e['name'] not in ['value', 'function_body']:
            return     
    replot(
        vertical_position=vertical_position_widget.value, 
        horizontal_v=horizontal_v_widget.value,
        vertical_v=vertical_v_widget.value,
    )
    
    # Print info on the "correctness" of the user's function
    check_user_value()
    
# Bind the sliders to the event
vertical_position_widget.observe(recompute)
horizontal_v_widget.observe(recompute)
vertical_v_widget.observe(recompute)

# Bind also the code widget
code_widget.observe(recompute)

In [ ]:
# Perform the first recomputation (to create the plot)
_ = recompute(None)